In [1]:
!pip install gpytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import gpytorch
from gpytorch.models import ApproximateGP
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import time
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")





In [4]:

df = pd.read_csv('/content/kathmandu_historical_air_quality_20250825_062407.csv')
df['datetime'] = pd.to_datetime(df['datetime'])

In [5]:


df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

In [6]:

le_station = LabelEncoder()
df['station_encoded'] = le_station.fit_transform(df['station_id'])
le_season = LabelEncoder()
df['season_encoded'] = le_season.fit_transform(df['season'])


In [7]:
features = [
    'temperature', 'humidity', 'pressure', 'wind_speed', 'wind_direction',
    'precipitation', 'lat', 'lng', 'elevation', 'aod_550nm',
    'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
    'weekday', 'is_weekend', 'high_pollution_season', 'festival_season',
    'station_encoded', 'season_encoded'
]


In [8]:
X = df[features].values
y = df['pm2_5'].values
X = np.nan_to_num(X, nan=np.nanmean(X, axis=0))

sort_idx = df['datetime'].argsort()
X_sorted, y_sorted = X[sort_idx], y[sort_idx]
df_sorted = df.iloc[sort_idx].reset_index(drop=True)

n_total = len(X_sorted)
n_train = int(0.70 * n_total)
n_val = int(0.15 * n_total)

X_train = X_sorted[:n_train]
y_train = y_sorted[:n_train]
X_val = X_sorted[n_train:n_train+n_val]
y_val = y_sorted[n_train:n_train+n_val]
X_test = X_sorted[n_train+n_val:]
y_test = y_sorted[n_train+n_val:]

scaler_X, scaler_y = StandardScaler(), StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
X_val_scaled = scaler_X.transform(X_val)
y_val_scaled = scaler_y.transform(y_val.reshape(-1, 1)).ravel()
X_test_scaled = scaler_X.transform(X_test)

df_test = df_sorted.iloc[n_train+n_val:].copy()

In [ ]:
class SparseGPModel(ApproximateGP):
    def __init__(self, inducing_points, num_dims):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(
            inducing_points.size(0)
        )
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(ard_num_dims=num_dims)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_sparse_gp(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled,
                   n_inducing=1200, num_epochs=120):
    kmeans = KMeans(n_clusters=n_inducing, random_state=42, n_init=10)
    inducing_points = torch.tensor(
        kmeans.fit(X_train_scaled).cluster_centers_, dtype=torch.float32
    ).to(device)

    model = SparseGPModel(inducing_points, X_train_scaled.shape[1]).to(device)
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)

    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam([
        {'params': model.parameters(), 'lr': 0.01},
        {'params': likelihood.parameters(), 'lr': 0.01},
    ], weight_decay=1e-4)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', patience=10, factor=0.7, min_lr=1e-5
    )

    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=len(y_train_scaled))

    X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(device)
    X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val_scaled, dtype=torch.float32).to(device)

    best_val_loss = float('inf')
    patience_counter = 0
    batch_size = 1024

    start_time = time.time()

    for epoch in range(num_epochs):
        model.train()
        likelihood.train()
        epoch_loss = 0

        for i in range(0, len(X_train_tensor), batch_size):
            batch_x = X_train_tensor[i:i+batch_size]
            batch_y = y_train_tensor[i:i+batch_size]

            optimizer.zero_grad()
            output = model(batch_x)
            loss = -mll(output, batch_y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()

        model.eval()
        likelihood.eval()
        with torch.no_grad():
            val_output = model(X_val_tensor)
            val_loss = -mll(val_output, y_val_tensor).item()

        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = {
                'model': model.state_dict(),
                'likelihood': likelihood.state_dict(),
            }
        else:
            patience_counter += 1

        if patience_counter >= 15:
            break

    if 'best_model_state' in locals():
        model.load_state_dict(best_model_state['model'])
        likelihood.load_state_dict(best_model_state['likelihood'])

    training_time = time.time() - start_time
    return model, likelihood, training_time

gp_model, gp_likelihood, gp_training_time = train_sparse_gp(
    X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled
)

In [ ]:
baseline_models = {}
baseline_times = {}

start_time = time.time()
rf_model = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1,
                                max_depth=20, min_samples_split=5)
rf_model.fit(X_train_scaled, y_train_scaled)
baseline_models['Random Forest'] = rf_model
baseline_times['Random Forest'] = time.time() - start_time

start_time = time.time()
gb_model = GradientBoostingRegressor(n_estimators=200, random_state=42,
                                   learning_rate=0.1, max_depth=6)
gb_model.fit(X_train_scaled, y_train_scaled)
baseline_models['Gradient Boosting'] = gb_model
baseline_times['Gradient Boosting'] = time.time() - start_time

start_time = time.time()
svr_model = SVR(kernel='rbf', C=10, gamma='scale', epsilon=0.1)
svr_model.fit(X_train_scaled, y_train_scaled)
baseline_models['SVR'] = svr_model
baseline_times['SVR'] = time.time() - start_time

start_time = time.time()
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train_scaled)
baseline_models['Linear Regression'] = lr_model
baseline_times['Linear Regression'] = time.time() - start_time

In [ ]:
def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / np.maximum(y_true, 1e-8))) * 100
    return {'RMSE': rmse, 'MAE': mae, 'R²': r2, 'MAPE': mape}

gp_model.eval()
gp_likelihood.eval()
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

with torch.no_grad():
    gp_pred_dist = gp_likelihood(gp_model(X_test_tensor))
    gp_pred_scaled = gp_pred_dist.mean.cpu().numpy()
    gp_var_scaled = gp_pred_dist.variance.cpu().numpy()

gp_pred = scaler_y.inverse_transform(gp_pred_scaled.reshape(-1, 1)).ravel()
gp_std = np.sqrt(gp_var_scaled) * scaler_y.scale_
y_test_orig = scaler_y.inverse_transform(scaler_y.transform(y_test.reshape(-1, 1))).ravel()

baseline_preds = {}
for name, model in baseline_models.items():
    pred_scaled = model.predict(X_test_scaled)
    pred_orig = scaler_y.inverse_transform(pred_scaled.reshape(-1, 1)).ravel()
    baseline_preds[name] = pred_orig

gp_metrics = calculate_metrics(y_test_orig, gp_pred)
residuals = np.abs(y_test_orig - gp_pred)
within_1std = (residuals <= gp_std).mean() * 100
gp_metrics['Uncertainty Calib'] = within_1std

baseline_metrics = {}
for name, pred in baseline_preds.items():
    baseline_metrics[name] = calculate_metrics(y_test_orig, pred)

In [ ]:
best_baseline = min(baseline_metrics.keys(), key=lambda x: baseline_metrics[x]['RMSE'])
best_baseline_rmse = baseline_metrics[best_baseline]['RMSE']
gp_improvement = ((best_baseline_rmse - gp_metrics['RMSE']) / best_baseline_rmse) * 100

df_test['gp_predictions'] = gp_pred
seasonal_performance = {}
for season in df_test['season'].unique():
    season_mask = df_test['season'] == season
    if season_mask.sum() > 0:
        season_data = df_test[season_mask]
        season_rmse = np.sqrt(mean_squared_error(season_data['pm2_5'], season_data['gp_predictions']))
        season_r2 = r2_score(season_data['pm2_5'], season_data['gp_predictions'])
        seasonal_performance[season] = {
            'RMSE': season_rmse,
            'R²': season_r2,
            'Count': season_mask.sum(),
            'Mean_PM25': season_data['pm2_5'].mean()
        }

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

models = ['Sparse GP'] + list(baseline_metrics.keys())
rmse_values = [gp_metrics['RMSE']] + [baseline_metrics[m]['RMSE'] for m in baseline_metrics.keys()]
colors = ['red'] + ['lightblue'] * (len(models) - 1)
bars = axes[0,0].bar(models, rmse_values, color=colors, alpha=0.7)
axes[0,0].set_title('RMSE Comparison Across All Models')
axes[0,0].set_ylabel('RMSE (μg/m³)')
axes[0,0].tick_params(axis='x', rotation=45)
for bar, val in zip(bars, rmse_values):
    axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
                   f'{val:.1f}', ha='center', va='bottom', fontsize=9)

scatter = axes[0,1].scatter(y_test_orig, gp_pred, alpha=0.6, s=15, c=gp_std, cmap='viridis')
min_val = min(y_test_orig.min(), gp_pred.min())
max_val = max(y_test_orig.max(), gp_pred.max())
axes[0,1].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
axes[0,1].set_xlabel('Actual PM2.5 (μg/m³)')
axes[0,1].set_ylabel('Predicted PM2.5 (μg/m³)')
axes[0,1].set_title(f'GP Predictions (R² = {gp_metrics["R²"]:.3f})')
plt.colorbar(scatter, ax=axes[0,1], label='Uncertainty (μg/m³)')

r2_values = [gp_metrics['R²']] + [baseline_metrics[m]['R²'] for m in baseline_metrics.keys()]
axes[0,2].bar(models, r2_values, color=colors, alpha=0.7)
axes[0,2].set_title('R² Comparison Across All Models')
axes[0,2].set_ylabel('R² Score')
axes[0,2].tick_params(axis='x', rotation=45)
for i, val in enumerate(r2_values):
    axes[0,2].text(i, val + 0.01, f'{val:.3f}', ha='center', va='bottom', fontsize=9)

n_show = min(300, len(y_test_orig))
x_time = range(n_show)
axes[1,0].plot(x_time, y_test_orig[:n_show], 'b-', label='Actual', alpha=0.8, linewidth=1)
axes[1,0].plot(x_time, gp_pred[:n_show], 'r-', label='GP Predicted', linewidth=1)
axes[1,0].fill_between(x_time,
                      (gp_pred - 1.96*gp_std)[:n_show],
                      (gp_pred + 1.96*gp_std)[:n_show],
                      alpha=0.3, color='red', label='95% CI')
axes[1,0].set_xlabel('Time Index')
axes[1,0].set_ylabel('PM2.5 (μg/m³)')
axes[1,0].set_title('Time Series with Uncertainty Bands')
axes[1,0].legend()

seasons = list(seasonal_performance.keys())
season_rmse = [seasonal_performance[s]['RMSE'] for s in seasons]
season_colors = ['brown', 'green', 'orange', 'blue'][:len(seasons)]
axes[1,1].bar(seasons, season_rmse, color=season_colors, alpha=0.7)
axes[1,1].set_title('GP Performance by Season')
axes[1,1].set_ylabel('RMSE (μg/m³)')
axes[1,1].tick_params(axis='x', rotation=45)

feature_importance = rf_model.feature_importances_
top_indices = np.argsort(feature_importance)[-10:]
axes[1,2].barh(range(len(top_indices)), feature_importance[top_indices])
axes[1,2].set_yticks(range(len(top_indices)))
axes[1,2].set_yticklabels([features[i] for i in top_indices])
axes[1,2].set_xlabel('Feature Importance (RF)')
axes[1,2].set_title('Top 10 Most Important Features')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Data for plotting
models = ['Sparse GP'] + list(baseline_metrics.keys())
rmse_values = [gp_metrics['RMSE']] + [baseline_metrics[m]['RMSE'] for m in baseline_metrics.keys()]
colors = ['red'] + ['lightblue'] * (len(models) - 1)

# Create plot
plt.figure(figsize=(10, 6))
bars = plt.bar(models, rmse_values, color=colors, alpha=0.7)
plt.title('RMSE Comparison Across All Models')
plt.ylabel('RMSE (μg/m³)')
plt.xticks(rotation=45, ha='right')
for bar, val in zip(bars, rmse_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
             f'{val:.1f}', ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# GP prediction uncertanity
import numpy as np

# Data for plotting
min_val = min(y_test_orig.min(), gp_pred.min())
max_val = max(y_test_orig.max(), gp_pred.max())

# Create plot
plt.figure(figsize=(8, 8))
scatter = plt.scatter(y_test_orig, gp_pred, alpha=0.6, s=15, c=gp_std, cmap='viridis')
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
plt.xlabel('Actual PM2.5 (μg/m³)')
plt.ylabel('Predicted PM2.5 (μg/m³)')
plt.title(f'GP Predictions (R² = {gp_metrics["R²"]:.3f})')
plt.colorbar(scatter, label='Uncertainty (μg/m³)')
plt.tight_layout()
plt.show()

In [ ]:
# model comparison
# Data for plotting
models = ['Sparse GP'] + list(baseline_metrics.keys())
r2_values = [gp_metrics['R²']] + [baseline_metrics[m]['R²'] for m in baseline_metrics.keys()]
colors = ['red'] + ['lightblue'] * (len(models) - 1)

# Create plot
plt.figure(figsize=(10, 6))
bars = plt.bar(models, r2_values, color=colors, alpha=0.7)
plt.title('R² Comparison Across All Models')
plt.ylabel('R² Score')
plt.xticks(rotation=45, ha='right')
for i, val in enumerate(r2_values):
    plt.text(i, val + 0.01, f'{val:.3f}', ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# time series with uncertainity plot

# Data for plotting
n_show = min(300, len(y_test_orig))
x_time = range(n_show)

# Create plot
plt.figure(figsize=(12, 6))
plt.plot(x_time, y_test_orig[:n_show], 'b-', label='Actual', alpha=0.8, linewidth=1)
plt.plot(x_time, gp_pred[:n_show], 'r-', label='GP Predicted', linewidth=1)
plt.fill_between(x_time,
                 (gp_pred - 1.96*gp_std)[:n_show],
                 (gp_pred + 1.96*gp_std)[:n_show],
                 alpha=0.3, color='red', label='95% CI')
plt.xlabel('Time Index')
plt.ylabel('PM2.5 (μg/m³)')
plt.title('Time Series with Uncertainty Bands')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# sesonal variation
# Data for plotting
seasons = list(seasonal_performance.keys())
season_rmse = [seasonal_performance[s]['RMSE'] for s in seasons]
season_colors = ['brown', 'green', 'orange', 'blue'][:len(seasons)]

# Create plot
plt.figure(figsize=(10, 6))
plt.bar(seasons, season_rmse, color=season_colors, alpha=0.7)
plt.title('GP Performance by Season')
plt.ylabel('RMSE (μg/m³)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# feature importance plot
# Data for plotting
feature_importance = rf_model.feature_importances_
top_indices = np.argsort(feature_importance)[-10:]

# Create plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(top_indices)), feature_importance[top_indices])
plt.yticks(range(len(top_indices)), [features[i] for i in top_indices])
plt.xlabel('Feature Importance (RF)')
plt.title('Top 10 Most Important Features')
plt.tight_layout()
plt.show()